In [1]:
# Giorgi Alavidze, Feb 22
# The notebook includes the following:
# (1) "Optimal" brute force search
# (2) Hashing 
# (3) Small essay on why Hashing defeats the brute force search (1)
# (4) K-anonymized dataset 


In [2]:
import requests 
import json 
import pandas as pd
import itertools
import pandasql
import os
import matplotlib
import numpy as np 
import secrets
mil = 1000000

In [3]:
def open_csv():
    '''
    The CSV file is attached on github
    '''
    df = pd.read_csv('lottoproject.csv')
    df = df.drop(df.columns[0],axis=1) #get rid of unwanted columns
    return df

In [4]:
nunique = [
    ['თბილისი','010'],
    ['ლანჩხუთი','260'],
    ['ბათუმი','610'],
    ['ქუთაისი','600'],
    ['მცხეთა','310'],
    ['საჩხერე','380'],
    ['ახალციხე','470'],
    ['საგარეჯო','360'],
    ['ჩოხატაური','460'],
    ['წყალტუბო','530'],
    ['ზუგდიდი','190'],
    ['ფოთი','420'],
    ['ქობულეთი','610'],
    ['ოზურგეთი','330'],
    ['მარნეული','280'],
    ['რუსთავი','350'],
    ['შუახევი','610'],
    ['გარდაბანი','120'],
    ['თელავი','200'],
    ['ნინოწმინდა','320'],
    ['გორი','590'],
    ['ახალქალაქი','070'],
    ['სენაკი','390'],
    ['ქედა','610'],
    ['გურჯაანი','130'],
    ['ჩხოროწყუ','480'],
    ['ონი','340'],
    ['ქარელი','430'],
    ['ბაღდათი','090'],
    ['კასპი','240'],
    ['ზესტაფონი','180'],
    ['ყვარელი','450'],
    ['ადიგენი','030'],
    ['თერჯოლა','210'],
    ['ვანი','170'],
    ['ცხინვალი','500'],
    ['დუშეთი','160'],
    ['დედოფლისწყარო','140'],
    ['ხულო','660'],
    ['ჭიათურა','540'],
    ['სამტრედია','370'],
    ['ხობი','580'],
    ['ხაშური','570'],
    ['ლაგოდეხი','250'],
    ['ახმეტა','080'],
    ['ცაგერი','490'],
    ['ბორჯომი','110'],
    ['ყაზბეგი','440'],
    ['ბოლნისი','100'],
    ['სიღნაღი','400'],
    ['მესტია','300'],
    ['ასპინძა','050'],
    ['ხელვაჩაური','610'],
    ['ამბროლაური','040'],
    ['თეთრიწყარო','220'],
    ['ტყიბული','410'],
    ['თიანეთი','230'],
    ['ხონი','550'],
    ['ლენტეხი','270'],
    ['ხარაგაული','560'],
    ['მარტვილი','290'],
    ['დმანისი','150'],
    ['აბაშა','020'],
    ['წალენჯიხა','510'],
    ['წალკა','520'],
    ['ახალგორი','060'],
    ['თბილისი','003'],
    ['თბილისი','004'],
    ['თბილისი','005'],
    ['თბილისი','006'],
    ['თბილისი','007'],
    ['თბილისი','008'],
    ['თბილისი','009'],
    ['თბილისი','011'],
    ['თბილისი','012'],
    ['ამბროლაური','013'],
    ['ზუგდიდი','014'],
    ['თელავი','015'],
    ['მცხეთა','016'],
    ['ოზურგეთი','017'],
    ['რუსთავი','018'],
    ['ახალციხე','019'],
    ['გორი','020'],
    ['ქუთაისი','021'],
    ['ბათუმი','022'],
    ['OCCUPIEDABKH','023'],
    ['მარტვილი','291'],
    ['ადიგენი','033'],
    ['OCCUPIEDABKH','650']
]

res = dict()
for sub in nunique:
    res[(sub[1])] = (sub[0])

df = open_csv()
#add born as column
pid = df['personalId']
born = []
abh = 0
pid_pre = [i[:3] for i in pid]
flag = False
yo = 0
for i in pid_pre:
    flag = False
    if(i == '620'):
        born.append('OCCUPIED')
        flag = True
    for k in res.keys():
        if(i == k):
            born_len = len(born)
            born.append(res[k])
            if(born_len == len(born)):
                abh += 1       
            flag = True
            
    if(not flag):
        born.append('blank')

df['Born'] = born



In [5]:
df

,personalId,wonAmount,initials,vaccineName,vaccinationPlace,Born
0,260-----873,500,ი. კ.,Sinopharm,ლანჩხუთი,ლანჩხუთი
1,010-----912,500,თ. კ.,Pfizer,თბილისი,თბილისი
2,330-----012,500,ჯ. ს.,Sinopharm,ბათუმი,ოზურგეთი
3,010-----773,500,თ. ხ.,Pfizer,თბილისი,თბილისი
4,600-----860,500,ი. ვ.,Pfizer,ქუთაისი,ქუთაისი
...,...,...,...,...,...,...
18329,600-----474,1000,ნ. მ.,Pfizer,ქუთაისი,ქუთაისი
18330,170-----445,1000,ს. ჭ.,Sinopharm,თბილისი,ვანი
18331,330-----794,1000,კ. ბ.,Sinopharm,ოზურგეთი,ოზურგეთი
18332,570-----164,1000,ს. ქ.,Pfizer,თბილისი,ხაშური


In [6]:

df['vaccineName'].value_counts()
u_born = list(df['Born'].unique())

born_unique = u_born
vacc_types = ['Sinopharm','Pfizer','SINOVAC','AstraZeneca']
sizes = []
dic = {}
vaccName_dic = {'0-10': [], '11-50': [], '51-100': [], '101-1000': [], '1000+': []}
for vacc in vacc_types:
        for born in u_born:
                x = df[((df["Born"]==born) & (df["vaccineName"]==vacc))].shape[0]
                sizes.append(x)
zero_ten = 0
ten_fifty = 0
fifty_hun = 0
hun_thousand = 0
thousandp = 0
for i in sizes:
        if(i < 10):
                zero_ten += 1
        elif(i < 50):
                ten_fifty += 1
        elif(i < 100):
                fifty_hun += 1
        elif(i < 1000):
                hun_thousand += 1
        else:
                thousandp += 1

vaccName_dic['0-10'] = zero_ten
vaccName_dic['11-50'] = ten_fifty
vaccName_dic['51-100'] = fifty_hun
vaccName_dic['101-1000'] = hun_thousand
vaccName_dic['1000+'] = thousandp

print(vaccName_dic)

{'0-10': 90, '11-50': 95, '51-100': 38, '101-1000': 26, '1000+': 3}


In [7]:
born_unique = u_born
u_vaccplace = list(df['vaccinationPlace'].unique())
vacc_types = ['Sinopharm','Pfizer','SINOVAC','AstraZeneca']
sizes = []
dic = {}
vaccPlace_dic = {'0-10': [], '11-50': [], '51-100': [], '101-1000': [], '1000+': []}
for born in u_born:
   for vacc in u_vaccplace:
        x = df[((df["Born"]==born) & (df["vaccinationPlace"]==vacc))].shape[0]
        sizes.append(x)

zero_ten = 0
ten_fifty = 0
fifty_hun = 0
hun_thousand = 0
thousandp = 0
for i in sizes:
        if(i < 10):
                zero_ten += 1
        elif(i < 50):
                ten_fifty += 1
        elif(i < 100):
                fifty_hun += 1
        elif(i < 1000):
                hun_thousand += 1
        else:
                thousandp += 1

vaccPlace_dic['0-10'] = zero_ten
vaccPlace_dic['11-50'] = ten_fifty
vaccPlace_dic['51-100'] = fifty_hun
vaccPlace_dic['101-1000'] = hun_thousand
vaccPlace_dic['1000+'] = thousandp
vaccPlace_dic

{'0-10': 3973, '11-50': 128, '51-100': 32, '101-1000': 23, '1000+': 2}

In [8]:
#RAW vacctype & born
B = 1 #budget
born_unique = u_born
vacc_types = ['Born','Sinopharm','Pfizer','SINOVAC','AstraZeneca']
lis_city = []
lis_city_lap = []
for born in born_unique:
       #RAW
       sino = len(df.loc[(df['Born'] == born) & (df['vaccineName'] == 'Sinopharm')])
       pfi = len(df.loc[(df['Born'] == born) & (df['vaccineName'] == 'Pfizer')])
       svac = len(df.loc[(df['Born'] == born) & (df['vaccineName'] == 'SINOVAC')])
       az = len(df.loc[(df['Born'] == born) & (df['vaccineName'] == 'AstraZeneca')])
       
       total = sino + pfi + svac + az
       sino_p = round(sino * 100 / total,2)
       pfi_p = round(pfi * 100 / total,2)
       svac_p = round(svac * 100 / total,2)
       az_p = round(az * 100 / total,2)
       lis_city.append([born,sino_p,pfi_p,svac_p,az_p])

       #DP
       e = B/3

       sino_dp = sino + np.random.laplace(0,1/e)
       pfi_dp = pfi + np.random.laplace(0,1/e)
       svac_dp = svac + np.random.laplace(0,1/e)
       az_dp = az + np.random.laplace(0,1/e)

       if(sino_dp < 0): sino_dp = 0
       if(pfi_dp < 0): pfi_dp = 0
       if(svac_dp < 0): svac_dp = 0
       if(az_dp < 0): az_dp = 0

       total_dp = sino_dp + pfi_dp + svac_dp + az_dp    

       sino_dp_p = round(sino_dp * 100 / total_dp,2)
       pfi_dp_p = round(pfi_dp * 100 / total_dp,2)
       svac_dp_p = round(svac_dp * 100 / total_dp,2)
       az_dp_p = round(az_dp * 100 / total_dp,2)

       lis_city_lap.append([born,sino_dp_p,pfi_dp_p,svac_dp_p,az_dp_p])


df_raw_vacctype = pd.DataFrame(columns=vacc_types)
df_raw_vacctype = df_raw_vacctype.append(pd.DataFrame(lis_city, columns = df_raw_vacctype.columns))
df_raw_vacctype

/var/folders/1c/6029jhp15w159bt1z4pc92br0000gn/T/ipykernel_18352/2662214462.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw_vacctype = df_raw_vacctype.append(pd.DataFrame(lis_city, columns = df_raw_vacctype.columns))


,Born,Sinopharm,Pfizer,SINOVAC,AstraZeneca
0,ლანჩხუთი,46.51,38.37,12.21,2.91
1,თბილისი,30.28,51.61,8.28,9.83
2,ოზურგეთი,35.38,54.87,7.18,2.56
3,ქუთაისი,23.8,66.24,6.47,3.49
4,blank,17.72,74.37,6.96,0.95
...,...,...,...,...,...
58,ყაზბეგი,25.0,65.0,10.0,0.0
59,OCCUPIEDABKH,26.67,66.67,0.0,6.67
60,ახალგორი,40.0,60.0,0.0,0.0
61,წალენჯიხა,32.88,50.68,16.44,0.0


In [9]:
#DP vacctype & born
df_DP_vacctype = pd.DataFrame(columns=vacc_types)
df_DP_vacctype = df_DP_vacctype.append(pd.DataFrame(lis_city_lap,columns = df_DP_vacctype.columns))
df_DP_vacctype


/var/folders/1c/6029jhp15w159bt1z4pc92br0000gn/T/ipykernel_18352/597194415.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_DP_vacctype = df_DP_vacctype.append(pd.DataFrame(lis_city_lap,columns = df_DP_vacctype.columns))


,Born,Sinopharm,Pfizer,SINOVAC,AstraZeneca
0,ლანჩხუთი,51.64,38.77,9.37,0.22
1,თბილისი,30.18,51.63,8.29,9.91
2,ოზურგეთი,35.17,54.26,6.66,3.91
3,ქუთაისი,23.87,66.22,6.1,3.81
4,blank,16.7,74.32,8.59,0.39
...,...,...,...,...,...
58,ყაზბეგი,22.19,34.15,43.65,0.0
59,OCCUPIEDABKH,19.92,80.08,0.0,0.0
60,ახალგორი,51.99,29.36,0.0,18.66
61,წალენჯიხა,31.08,53.73,15.19,0.0


In [10]:
#RAW vacctype & born
B = 1 #budget
vaccplace_unique = df['vaccinationPlace'].unique()
vacc_types = ['vaccPlace','Sinopharm','Pfizer','SINOVAC','AstraZeneca']
lis_city = []
lis_city_lap = []
for vaccplace in vaccplace_unique:
       #RAW
       sino = len(df.loc[(df['vaccinationPlace'] == vaccplace) & (df['vaccineName'] == 'Sinopharm')])
       pfi = len(df.loc[(df['vaccinationPlace'] == vaccplace) & (df['vaccineName'] == 'Pfizer')])
       svac = len(df.loc[(df['vaccinationPlace'] == vaccplace) & (df['vaccineName'] == 'SINOVAC')])
       az = len(df.loc[(df['vaccinationPlace'] == vaccplace) & (df['vaccineName'] == 'AstraZeneca')])
       
       total = sino + pfi + svac + az
       sino_a = round(sino * 100 / total,2)
       pfi_a = round(pfi * 100 / total,2)
       svac_a = round(svac * 100 / total,2)
       az_a = round(az * 100 / total,2)
       lis_city.append([vaccplace,sino_a,pfi_a,svac_a,az_a])

       #DP
       e = B/3

       sino_dp = sino + np.random.laplace(0,1/e)
       pfi_dp = pfi + np.random.laplace(0,1/e)
       svac_dp = svac + np.random.laplace(0,1/e)
       az_dp = az + np.random.laplace(0,1/e)

       if(sino_dp < 0): sino_dp = 0
       if(pfi_dp < 0): pfi_dp = 0
       if(svac_dp < 0): svac_dp = 0
       if(az_dp < 0): az_dp = 0

       total_dp = sino_dp + pfi_dp + svac_dp + az_dp    

       sino_dp_p = round(sino_dp * 100 / total_dp,2)
       pfi_dp_p = round(pfi_dp * 100 / total_dp,2)
       svac_dp_p = round(svac_dp * 100 / total_dp,2)
       az_dp_p = round(az_dp * 100 / total_dp,2)

       lis_city_lap.append([vaccplace,sino_dp_p,pfi_dp_p,svac_dp_p,az_dp_p])

df_raw_vacctype_2 = pd.DataFrame(columns=vacc_types)
df_raw_vacctype_2 = df_raw_vacctype_2.append(pd.DataFrame(lis_city, columns = df_raw_vacctype_2.columns))
df_raw_vacctype_2

/var/folders/1c/6029jhp15w159bt1z4pc92br0000gn/T/ipykernel_18352/1774035394.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw_vacctype_2 = df_raw_vacctype_2.append(pd.DataFrame(lis_city, columns = df_raw_vacctype_2.columns))


,vaccPlace,Sinopharm,Pfizer,SINOVAC,AstraZeneca
0,ლანჩხუთი,68.42,15.79,15.79,0.0
1,თბილისი,27.49,56.8,7.38,8.34
2,ბათუმი,23.66,65.37,7.46,3.51
3,ქუთაისი,24.17,64.71,7.26,3.86
4,მცხეთა,22.14,62.14,9.29,6.43
...,...,...,...,...,...
61,დმანისი,23.08,76.92,0.0,0.0
62,აბაშა,70.27,0.0,27.03,2.7
63,წალენჯიხა,48.98,34.69,16.33,0.0
64,წალკა,34.78,26.09,39.13,0.0


In [11]:
#DP vacctype & born
df_DP_vacctype = pd.DataFrame(columns=vacc_types)
df_DP_vacctype.append(pd.DataFrame(lis_city_lap,columns = vacc_types))

/var/folders/1c/6029jhp15w159bt1z4pc92br0000gn/T/ipykernel_18352/1107079578.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_DP_vacctype.append(pd.DataFrame(lis_city_lap,columns = vacc_types))


,vaccPlace,Sinopharm,Pfizer,SINOVAC,AstraZeneca
0,ლანჩხუთი,69.57,13.7,14.91,1.82
1,თბილისი,27.5,56.81,7.31,8.37
2,ბათუმი,23.64,65.25,7.75,3.37
3,ქუთაისი,24.32,64.99,6.96,3.74
4,მცხეთა,19.45,63.03,11.23,6.3
...,...,...,...,...,...
61,დმანისი,44.47,54.79,0.74,0.0
62,აბაშა,64.04,0.0,35.96,0.0
63,წალენჯიხა,53.52,29.22,17.26,0.0
64,წალკა,26.5,35.79,37.7,0.0


In [12]:
#RAW vacctype & born
B = 1 #budget
vaccplace_unique
born_unique
lis_city = []
lis_city_p = []
lis_city_dp = []
total = 0
df_raw_born_vaccplace = pd.DataFrame(columns = born_unique)
df_dp_born_vaccplace = pd.DataFrame(columns = born_unique)

for born in born_unique:
    lis_city_p = []
    lis_city = []
    lis_city_dp = []

    inc = 63
    for vaccplace in vaccplace_unique:
        inc -= 1
        vaccplace_len = len(df.loc[(df['vaccinationPlace'] == vaccplace) & (df['Born'] == born)])
        total += vaccplace_len
        lis_city.append(vaccplace_len)
        dp_vaccplace_len = vaccplace_len + np.random.laplace(0,1/e)
        if(dp_vaccplace_len < 0 ): dp_vaccplace_len = 0
        lis_city_dp.append(dp_vaccplace_len)
        if(inc == 0):
            lis_city_p = [round(i * 100 / total,2) for i in lis_city]
            df_raw_born_vaccplace.loc[len(df_raw_born_vaccplace)] = lis_city_p
            lis_city_p = [round(i * 100 / total,2) for i in lis_city_dp]
            df_dp_born_vaccplace.loc[len(df_dp_born_vaccplace)] = lis_city_p
            lis_city_dp = []
            lis_city_p_df = []
        
            
            
    
    
    total = 0


In [14]:
df_raw_born_vaccplace.to_csv("df_raw_born.csv")

In [17]:
df_dp_born_vaccplace.to_csv("df_dp_born_2.csv")